# Predicting House Prices

The purpose of this project is to build a machine learning model using linear regression to predict house prices. The dataset for this project is a collection of housing data from Ames, Iowa from 2006 to 2010. The dataset and its documentaiton can be [found here.](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627)

To begin, I will import the necessary files and libraries to start cleaning and building our model.

In [1]:
# importing cleaning/visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import machine learning libraries
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

# allowing for viewing of large file
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [2]:
# reading csv file to dataframe
housing = pd.read_csv('AmesHousing.tsv', delimiter='\t')

# Creating Functions

Now I will create a few basic functions for cleaning, transforming and modeling the data.

In [3]:
# def a function for cleaning the data
def transform_features(df):
    return df

In [4]:
# def a function for feature selection
def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

In [5]:
# def a function for training/testing
def train_and_test(df):
    # splitting dataframe into train/test segments
    train = df[0:1460]
    test = df[1460:]
    
    # selecting only numeric values from train/test sets
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

# Feature Engineering

Now I will create the internals for the feature engineering function. This function should do the following:

* Remove all features with more than 5% null values.
* For text columns, drop rows with missing values.
* Fill remaining null values in numerical columns with most common value.
* Convert text fields to categorical or drop them.
* Explore any features which need to be engineered for better performance.

### Drop Columns with >5% Missing Values

In [6]:
# exploring original dataframe
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [7]:
# creating a df of null counts in each category
null_counts = housing.isnull().sum()

# identifying columns with more than 5% null values
drop_na_cols = null_counts[(null_counts > len(housing)/20)].sort_values()

# drop those columns from dataframe
housing = housing.drop(drop_na_cols.index, axis=1)

### For text columns, drop columns with missing values

In [8]:
# identifying only text columns
missing_text_vals = housing.select_dtypes(include=['object']).isnull().sum()
replace_text_cols = missing_text_vals[(missing_text_vals > 0)].sort_values()
replace_text_cols

Electrical         1
Mas Vnr Type      23
Bsmt Qual         80
Bsmt Cond         80
BsmtFin Type 1    80
BsmtFin Type 2    81
Bsmt Exposure     83
dtype: int64

In [9]:
# dropping columns with missing values in text columns
housing = housing.drop(replace_text_cols.index, axis=1)

### Fill Remaining Null Values in Numerical Columns With Most Common Value

In [10]:
# identifying only numerical columns
missing_vals = housing.select_dtypes(include=['int','float']).isnull().sum()
replace_cols = missing_vals[(missing_vals > 0)].sort_values()
replace_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

To learn more about these columns, I will consult the [documentation for the original dataset.](https://s3.amazonaws.com/dq-content/307/data_description.txt)

After reviewing these columns, I can determine which would be represented best by the mean (coninuous), and which would be best represented by the mode (categorical).

* BsmtFin SF 1: Finished Square Feet (Mean)
* BsmtFin SF 2: Type 2 Finished Square Feet (Mean)
* Bsmt Unf SF: Unfinished Basement Square Feet (Mean)
* Total Bsmt SF: Total SF of basement area (Mean)
* Garage Cars: Size of garage in car capacity (Mean)
* Garage Area: Size of garage in square feet (Mean)
* Bsmt Full Bath: Basement full bathrooms (Mean) 
* Bsmt Half Bath: Basement half bathrooms (Mean)
* Mas Vnr Area: Masonry veneer area (Mean)

For all of these columns it would make sense to replace null values with the mean.

In [11]:
## Fill missing values with mean
housing = housing.fillna(housing.mean())

Features to be engineered:
These can be combined into a more useful feature:
* Year Built         2930 non-null int64
* Year Remod/Add     2930 non-null int64

These can be combined into two more useful features:
* Bsmt Full Bath     2930 non-null float64
* Bsmt Half Bath     2930 non-null float64
* Full Bath          2930 non-null int64
* Half Bath          2930 non-null int64

These are proxies for sale, and should be deleted:
* Mo Sold            2930 non-null int64
* Yr Sold            2930 non-null int64
* Sale Type          2930 non-null object
* Sale Condition     2930 non-null object

In [12]:
# Engineering Age of house feature
years_sold = housing['Yr Sold'] - housing['Year Built']
# Dropping Year Built column
housing = housing.drop('Year Built', axis=1)
# Checking for Negative values
years_sold[years_sold<0]

2180   -1
dtype: int64

In [13]:
# Engineering Years Before Remodel column
housing['years_since_remodel'] = housing['Yr Sold'] - housing['Year Remod/Add']
# Dropping Year Remod column
housing = housing.drop('Year Remod/Add', axis=1)

In [14]:
# verifying that new feature contains positive values
years_since_remod = housing['years_since_remodel']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
Name: years_since_remodel, dtype: int64

In [15]:
# Creating new columns
housing['Years Before Sale'] = years_sold
housing['Years Since Remod'] = years_since_remod

# Drop rows with negative values
housing = housing.drop([2180, 1702, 2181], axis=0)

In [16]:
# Combining bathroom features
housing['Bath'] = housing['Full Bath'] + (housing['Half Bath']*0.5)
housing['Bsmt Bath'] = housing['Bsmt Full Bath'] + (housing['Bsmt Half Bath']*0.5)
# Dropping old bathroom features
housing = housing.drop(['Full Bath','Half Bath','Bsmt Full Bath','Bsmt Half Bath'], axis=1)

In [17]:
# Dropping columns which are proxy for sale
housing = housing.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'], axis=1)

In [18]:
# Dropping columns which aren't useful for our model
housing = housing.drop(['PID','Order'], axis=1)

Now I will take the existing code and add it back into my `'transform_features'` function for future efficiency.

In [19]:
# def a function for cleaning the data
def transform_features(df):
    # creating a df of null counts in each category
    null_counts = df.isnull().sum()
    # identifying columns with more than 5% null values
    drop_na_cols = null_counts[(null_counts > len(df)/20)].sort_values()
    # drop those columns from dataframe
    df.drop(drop_na_cols.index, axis=1, inplace=True)
    
    # identifying only text columns
    missing_text_vals = df.select_dtypes(include=['object']).isnull().sum()
    replace_text_cols = missing_text_vals[(missing_text_vals > 0)].sort_values()
    # dropping columns with missing values in text columns
    df.drop(replace_text_cols.index, axis=1, inplace=True)
    
    # identifying only numerical columns
    missing_vals = df.select_dtypes(include=['int','float']).isnull().sum()
    replace_cols = missing_vals[(missing_vals > 0)].sort_values()
    ## Fill missing values with mean
    df.fillna(df.mean(), inplace=True)
    
    # Engineering Age of house feature
    years_sold = df['Yr Sold'] - df['Year Built']
    # Dropping Year Built column
    df.drop('Year Built', axis=1, inplace=True)
    # Engineering Years Before Remodel column
    df['years_since_remodel'] = df['Yr Sold'] - df['Year Remod/Add']
    # Dropping Year Remod column
    df.drop('Year Remod/Add', axis=1, inplace=True)
    # verifying that new feature contains positive values
    years_since_remod = df['years_since_remodel']
    # Creating new columns
    df['Years Before Sale'] = years_sold
    df['Years Since Remod'] = years_since_remod
    # Drop rows with negative values
    df.drop([2180, 1702, 2181], axis=0, inplace=True)
    # Combining bathroom features
    df['Bath'] = df['Full Bath'] + (df['Half Bath']*0.5)
    df['Bsmt Bath'] = df['Bsmt Full Bath'] + (df['Bsmt Half Bath']*0.5)
    # Dropping old bathroom features
    df.drop(['Full Bath','Half Bath','Bsmt Full Bath','Bsmt Half Bath'], axis=1, inplace=True)
    # Dropping columns which are proxy for sale
    df.drop(['Mo Sold','Yr Sold','Sale Type','Sale Condition'], axis=1, inplace=True)
    # Dropping columns which aren't useful for our model
    df.drop(['PID','Order'], axis=1, inplace=True)
    
    return df

And I will run these functions with the current data to find an rmse prior to feature selection.

In [20]:
# Reading in original data into new df
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
# Transforming features
transform_features(data)
# Running regression model
train_and_test(data)

70888.75727098074

# Feature Selection

Now I will sort through the remaining features to see which features are highly correlated with `'SalePrice'` in an attempt to narrow down the dataset and make computation more efficient.

In [21]:
# Selecting only numerical columns
data_num = data.select_dtypes(include=['int','float'])
# Checking for correlation
data_corr_coef = data_num.corr()['SalePrice'].abs().sort_values(ascending=False)
data_corr_coef

SalePrice              1.000000
Overall Qual           0.801206
Gr Liv Area            0.717596
Garage Cars            0.648411
Total Bsmt SF          0.643601
Garage Area            0.641675
1st Flr SF             0.635185
Bath                   0.581357
Years Before Sale      0.558979
Years Since Remod      0.534985
years_since_remodel    0.534985
Mas Vnr Area           0.510611
TotRms AbvGrd          0.498574
Fireplaces             0.474831
BsmtFin SF 1           0.438928
Wood Deck SF           0.328183
Open Porch SF          0.316262
Bsmt Bath              0.270144
2nd Flr SF             0.269601
Lot Area               0.267520
Bsmt Unf SF            0.182248
Bedroom AbvGr          0.143916
Enclosed Porch         0.128685
Kitchen AbvGr          0.119760
Screen Porch           0.112280
Overall Cond           0.101540
MS SubClass            0.085128
Pool Area              0.068438
Low Qual Fin SF        0.037629
3Ssn Porch             0.032268
Misc Val               0.019273
BsmtFin 

In [22]:
# Keeping only columns with correlation coefficient larger than 0.4
data.drop(data_corr_coef[data_corr_coef < 0.4].index, axis=1, inplace=True)

Now I want to deal with categorical features that remain in the dataset. To do so, I will:
* Create a list of categorical column names
* Convert numerical features that should be categorical
* Drop categorical columns with more than 10 unique values
* Drop categorical columns with unique values under 2 and an very skewed distribution
* Create dummies for remaining categorical columns

### List of categorical features

Categorical string columns
* MS Zoning
* Street
* Lot Shape
* Land Contour
* Utilities
* Lot Config
* Land Slope
* Neighborhood
* Condition 1
* Condition 2
* Bldg Type
* House Style
* Roof Style
* Roof Matl
* Exterior 1st
* Exterior 2nd
* Foundation
* Heating
* Central Air
* Functional
* Paved Drive

Numerical features to be changed to `'category'` type:
* MS SubClass
* Exter Qual
* Exter Cond
* Heating QC
* Kitchen Qual

In [23]:
# Creating list of categorical features
cat_features = ['Street', 'Lot Shape', 'Land Contour', 'Utilities',
                'Lot Config', 'Land Slope', 'Neighborhood',
                'Condition 1', 'Condition 2', 'Bldg Type',
                'House Style', 'Roof Style','Roof Matl',
                'Exterior 1st','Exterior 2nd','Foundation',
                'Heating', 'Central Air', 'Functional', 'Paved Drive',
                'MS SubClass', 'Exter Qual', 'Exter Cond', 'Heating QC',
                'Kitchen Qual'
               ]

In [24]:
# Creating list of cat features left in dataset
transform_cat_features = []
for col in cat_features:
    if col in data.columns:
        transform_cat_features.append(col)
        
# Number of unique values in each cat column
unique_cat_vals = data[transform_cat_features].apply(lambda col: len(col.value_counts())).sort_values()
# Removing columns with more than 10 unique values
nonuniq_cols = unique_cat_vals[unique_cat_vals > 10].index
data.drop(nonuniq_cols, axis=1, inplace=True)

In [25]:
# Listing remaining cat columns for skewed data
remaining_cat_cols = []
for col in cat_features:
    if col in data.columns:
        remaining_cat_cols.append(col)
        
# Exploring remaining cat columns with less than 5 unique values
cat_val_counts = data[remaining_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
skewed_cols = cat_val_counts[cat_val_counts < 5].index
skewed_cols

Index(['Street', 'Central Air', 'Utilities', 'Land Slope', 'Paved Drive',
       'Lot Shape', 'Land Contour', 'Exter Qual'],
      dtype='object')

In [26]:
data['Street'].value_counts()

Pave    2915
Grvl      12
Name: Street, dtype: int64

In [27]:
data['Central Air'].value_counts()

Y    2731
N     196
Name: Central Air, dtype: int64

In [28]:
data['Utilities'].value_counts()

AllPub    2924
NoSewr       2
NoSeWa       1
Name: Utilities, dtype: int64

In [29]:
data['Land Slope'].value_counts()

Gtl    2786
Mod     125
Sev      16
Name: Land Slope, dtype: int64

In [30]:
data['Paved Drive'].value_counts()

Y    2649
N     216
P      62
Name: Paved Drive, dtype: int64

In [31]:
data['Lot Shape'].value_counts()

Reg    1859
IR1     976
IR2      76
IR3      16
Name: Lot Shape, dtype: int64

In [32]:
data['Land Contour'].value_counts()

Lvl    2632
HLS     120
Bnk     115
Low      60
Name: Land Contour, dtype: int64

In [33]:
data['Exter Qual'].value_counts()

TA    1799
Gd     988
Ex     105
Fa      35
Name: Exter Qual, dtype: int64

After viewing the value counts for these categories, it seems that `'Street'` and `'Utilities'` are highly skewed and should be dropped to prevent the algorithm from being skewed.

In [34]:
# dropping skewed columns
data.drop(['Street', 'Utilities'], axis=1, inplace=True)

In [35]:
# converting all cat columns that remain to categorical
# Creating list of cat features left in dataset
transform_cat_features = data.select_dtypes(include=['object'])

for col in transform_cat_features:
    data[col] = data[col].astype('category', inplace=True)

In [36]:
# creating drop list of category features
cat_cols = data.select_dtypes(include=['category'])

# creating df of dummy categories
dummies = pd.get_dummies(data.select_dtypes(include=['category']))

# Create dummies and add back to dataframe
data_final = pd.concat([
    data, dummies], axis=1)

# Drop original dummy categories
data_final.drop(cat_cols, axis=1, inplace=True)

Now, I will take this feature selection code and update the `'select_features'` function.

In [37]:
# def a function for feature selection
def select_features(df):
    ## Keeping correlated numerical features
    # Selecting only numerical columns
    df_num = df.select_dtypes(include=['int','float'])
    # Checking for correlation
    df_corr_coef = df_num.corr()['SalePrice'].abs().sort_values(ascending=False)
    df_corr_coef
    # Keeping only columns with correlation coefficient larger than 0.4
    df.drop(df_corr_coef[df_corr_coef < 0.4].index, axis=1, inplace=True)
    
    # Creating list of categorical features
    cat_features = ['Street', 'Lot Shape', 'Land Contour', 'Utilities',
                    'Lot Config', 'Land Slope', 'Neighborhood',
                    'Condition 1', 'Condition 2', 'Bldg Type',
                    'House Style', 'Roof Style','Roof Matl',
                    'Exterior 1st','Exterior 2nd','Foundation',
                    'Heating', 'Central Air', 'Functional', 'Paved Drive',
                    'MS SubClass', 'Exter Qual', 'Exter Cond', 'Heating QC',
                    'Kitchen Qual'
                   ]
    # Creating list of cat features left in dfset
    transform_cat_features = []
    for col in cat_features:
        if col in df.columns:
            transform_cat_features.append(col)
    
    ## Dropping cat features with too many unique values
    # Number of unique values in each cat column
    unique_cat_vals = df[transform_cat_features].apply(lambda col: len(col.value_counts())).sort_values()
    # Removing columns with more than 10 unique values
    nonuniq_cols = unique_cat_vals[unique_cat_vals > 10].index
    df.drop(nonuniq_cols, axis=1, inplace=True)
    
    ## Dropping cat features with skewed values
    # dropping skewed columns
    df.drop(['Street', 'Utilities'], axis=1, inplace=True)
    
    # converting all cat columns that remain to categorical
    # Creating list of cat features left in dfset
    transform_cat_features = df.select_dtypes(include=['object'])
    # Converting these features to category type
    for col in transform_cat_features:
        df[col] = df[col].astype('category', inplace=True)
    
    ## Get dummies for categorical features and drop them
    # creating drop list of category features
    cat_cols = df.select_dtypes(include=['category'])
    # creating df of dummy categories
    dummies = pd.get_dummies(df.select_dtypes(include=['category']))
    # Create dummies and add back to dfframe
    df_final = pd.concat([df, dummies], axis=1)
    # Drop original dummy categories
    df_final.drop(cat_cols, axis=1, inplace=True)

    return df_final

In [38]:
# Reading in original data into new df
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)
# Running regression model
train_and_test(data_final)

33297.34697167901

After feature selection, the RMSE has dropped considerably to $33,297

# Updating Model for K-Fold and Cross Validation

Now I will update the ML model to incorporate a new parameter which directs it to choose between holdout validation, simple cross validation, and k-fold cross validation.

In [48]:
# def a function for training/testing
def train_and_test(df, k=0):
    # splitting dataframe from target column
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop('SalePrice')
    lr=linear_model.LinearRegression()
    
    # holdout validation
    if k == 0:
        # splitting dataframe into train/test segments
        train = df[0:1460]
        test = df[1460:]
    
        lr.fit(train[features], train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test['SalePrice'], predictions)
        rmse = np.sqrt(mse) 
        return rmse
    
    # simple cross validation
    if k == 1:
        # Randomizing all rows from df
        shuffled_df = df.sample(frac=1, )
        # splitting dataframe into train/test segments
        train = df[0:1460]
        test = df[1460:]
        
        # training data
        lr.fit(train[features], train['SalePrice'])
        predictions_one = lr.predict(test[features])
        mse_one = mean_squared_error(test['SalePrice'], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        # test data
        lr.fit(test[features], test['SalePrice'])
        predictions_two = lr.predict(train[features])
        mse_two = mean_squared_error(train['SalePrice'], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        # averaging rmse
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one, rmse_two)
        return avg_rmse
    
    # K-fold validation
    else:
        # building K-folds
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            # creating train/test set for fold
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            # fitting and predicting
            lr.fit(train[features], train['SalePrice'])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test['SalePrice'], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        # Averaging rmse values
        avg_rmse = np.mean(rmse_values)
        print(avg_rmse)
        return avg_rmse

# Hyperparameter tuning

Finally, I will run the model several times with different k-values to find the lowest rmse and best fit.

## Holdout Testing

In [49]:
# Holdout testing
# Reading in original data into new df
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)
# Running regression model
train_and_test(data_final, k=0)

33297.34697167901

## Cross Validation Testing

In [50]:
# Simple cross validation
# Reading in original data into new df
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)
# Running regression model
train_and_test(data_final, k=1)

33297.34697167901 27135.94980576917


30216.64838872409

### K-Fold Testing

Finally, I will iterate through various K-Fold testing structures, with parameters from 2-100

In [51]:
# Reading in original data into new df
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')
# Transforming features
data_transformed = transform_features(data)
# Selecting features
data_final = select_features(data_transformed)

# testing to optimize k-fold value
k_range = list(range(2,100,1))
rmses = {}
for i in k_range:
    rmse = train_and_test(data_final, k=i)
    rmses[i] = rmse

30191.04657288207
29249.813491538225
29118.490442732815
1252279235.8764966
170729244.7830841
28583.0146041271
28729.95391732712
28700.884523504574
28282.52850884921
3808995.311976761
28382.13907635344
28376.787005710616
25080253.487318065
18089772.662302945
28042.81595897129
142848177.63732794
22512575.468647216
14727403.958816845
27915.300423656176
28014.120429004677
27801.648178092775
27757.00207917983
5086411.342394413
16153582.69100317
27832.57755481281
14500677.596383484
27637.28690366654
122237991.60079417
116941410.13315894
2072216.410895603
1850097.1089262904
62792018.29058874
27462.934000524696
53981046.06981623
9482063.750091715
27207.288800166083
27487.064651654786
5664495.685889551
27462.05430467067
53019609.07295413
27223.955488489315
9060564.70452755
11813965.612461569
27293.049668406133
27148.242717267993
27463.893553856346
27127.28845947485
26914.51865119825
27274.69843981179
26920.737003486945
214535.5061708254
26734.70454154348
26891.942657944706
26899.15784640637
396

In [52]:
# Using min() + list comprehension + values() 
# Finding min value keys in dictionary 
temp = min(rmses.values()) 
res = [key for key in rmses if rmses[key] == temp] 
  
# printing result  
print("Model with minimum rmse is: " + "k = " + str(res) + ", rmse = " + str(temp))

Model with minimum rmse is: k = [96], rmse = 26228.212563795645


# Final Results

In the end, the best model was achieved using K-Fold testing with 96 folds and a RMSE of $26,228.